# Artificial Neural Network

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Data Preprocessing

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [3]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [4]:
print(y)

[1 0 1 ... 1 1 0]


## Categorical Data Encoding

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [6]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### OneHotEncoding the "Geography" column

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [8]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Train Test Splitting

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

### Feature Scaling

In [10]:
from sklearn.preprocessing import RobustScaler
rsc = RobustScaler()
X_train = rsc.fit_transform(X_train)
X_test = rsc.transform(X_test)

## Building ANN

In [11]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding second hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding output layer

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

### Compiling the ANN

In [16]:
ann.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on Training set

In [17]:
ann.fit(X_train, y_train, batch_size= 32, epochs= 100)

Epoch 1/100
250/250 [==============================] - 1s 851us/step - loss: 0.5178 - accuracy: 0.7933
Epoch 2/100
250/250 [==============================] - 0s 847us/step - loss: 0.4658 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 859us/step - loss: 0.4414 - accuracy: 0.8098
Epoch 4/100
250/250 [==============================] - 0s 908us/step - loss: 0.4244 - accuracy: 0.8217
Epoch 5/100
250/250 [==============================] - 0s 855us/step - loss: 0.4137 - accuracy: 0.8249
Epoch 6/100
250/250 [==============================] - 0s 831us/step - loss: 0.4073 - accuracy: 0.8275
Epoch 7/100
250/250 [==============================] - 0s 823us/step - loss: 0.4023 - accuracy: 0.8291
Epoch 8/100
250/250 [==============================] - 0s 827us/step - loss: 0.3985 - accuracy: 0.8306
Epoch 9/100
250/250 [==============================] - 0s 831us/step - loss: 0.3950 - accuracy: 0.8296
Epoch 10/100
250/250 [==============================] - 0s 831us/step - l

## Making predictions and evaluating the model

### Predicting result of single observation

In [18]:
print(ann.predict(rsc.transform(
    [[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)


1/1 [==============================] - 0s 191ms/step
[[False]]


### Predicting Test set Results

In [20]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 645us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Confusion matrix

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)


[[1521   74]
 [ 198  207]]


0.864